In [1]:
import sys
sys.path.append('..')  
import _dgp as dg
from _metrics import  CrossValidation, accuracy_score
from  _logistic import LogisticRegression
import numpy as np
import pandas as pd


**In this notebook, I assume that the reader is already familiarized with the content available in "logit_binary.ipynb", so I will focus on explaining complementary information**

## **Data loading and preparation**

I generate data with 50 variables, among which there are 5 true variables and target variable with 4 classes

In [2]:
np.random.seed(0)
x,y=dg.gen_logistic_multiclass(n_classes=4)
columns=[f"column_{i}"for i in range(1,51)]
x.shape


(600, 50)

## **Estimation**

We can now apply simple Gradient descent optimisation algorithm to estimate B parameter of Logistic Regression 

In [3]:
model1=LogisticRegression(solver="gd",add_intercept=True,)
model1.fit(x,y)[0:5]

algorithm did not converge under 100 iterations,so the calculated parameter is biased


array([[-0.09695882,  0.02685312, -0.15198979],
       [ 0.61609505,  1.10368005,  1.41692107],
       [-1.07808941, -1.23254918,  0.31850209],
       [ 1.50002921, -0.370223  ,  0.7151769 ],
       [-0.66567419,  0.81889554, -0.55373456]])

**Algorithm could not converge:**
If you have this message, **NEVER** proceed to further steps as 
the estimated parameter is biased. 

Therefore, try other hyperparameters or optimisations to aim  convergence.


For example stochastic gradient descent with another learning rate

In [10]:
model2=LogisticRegression(solver="sgd",learning_rate=0.001,add_intercept=True,)
model2.fit(x,y)[0:5]

algorithm did not converge under 1900 iterations,so the calculated parameter is biased


array([[-0.09710782,  0.0275085 , -0.15131926],
       [ 0.61318401,  1.10223581,  1.41490585],
       [-1.0759883 , -1.23050519,  0.31862842],
       [ 1.49721833, -0.36942759,  0.71334461],
       [-0.66609153,  0.81788553, -0.55221879]])

Or Newton Raphson, but before doing it, please verify that the inverse of x.T@x exists

In [7]:
np.linalg.det(x.T@x)

4.498675345455717e+137

In [3]:
model3=LogisticRegression(solver="nr",learning_rate=0.001,add_intercept=True,)
print(model3.fit(x,y)[0:5])#viz only 5 first variables' coefficients for 3 classes (K-1 classes)
model3.params.shape

algorithm did  converge under 100 iterations (at 6 iterations)
[[ 0.13781676  0.34921783  0.15371239]
 [ 1.02871155  1.47233543  1.8281052 ]
 [-1.42982444 -1.54794253  0.06249892]
 [ 1.82262485 -0.21835224  0.97173556]
 [-0.82945255  0.76764172 -0.68815249]]


(51, 3)

**We have generated data with 4 classes, as we always one hot encode k-1 classes that is why we have 3 columns of B parameter and not 4**

## **Prediction**

The prediction is done by applying softmax activation function to the linear regression x@B:

In [4]:
predictions=model3.predict(x)
model3.proba[0:5]

array([[0.10463537, 0.48915723, 0.37662085, 0.02958655],
       [0.05650613, 0.03152616, 0.11391315, 0.79805455],
       [0.06054896, 0.00388308, 0.81882604, 0.11674192],
       [0.09734898, 0.25317433, 0.38832512, 0.26115157],
       [0.69978573, 0.00175103, 0.16246118, 0.13600206]])

And all probabilities sum up to 1

In [6]:
model3.proba[0:5].sum(1)

array([1., 1., 1., 1., 1.])

In order to use accuracy, we have to work not with y (real variable) but with model3.y ( one hot encoded version of y )

In [7]:
model3.y[0:5]

array([[1, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 0],
       [1, 0, 0]])

In [12]:
y[0:5]

array([0, 3, 2, 3, 0])

As you see, the first column is 0, second is 1, third is 2. This is done in the ascending order for logistic multiclass regression 

In [5]:
#nb of folds is 4 so we will run 4 models 
CV_scores=CrossValidation(Class_algorithm=model3,x=x,y=model3.y,metrics_function=accuracy_score,nb_k_fold=4)
print(CV_scores)
np.mean(CV_scores)

algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
[0.7266666666666667, 0.5466666666666666, 0.6266666666666667, 0.6666666666666666]


0.6416666666666667

## **Model Interpretability**

In [6]:
pd.set_option('display.max_rows', None)
model3.get_inference(only_IC=False,ordered_columns_names=columns)

,coef,std,z value,p value,p value star,odds ratio
intercept_0,0.1378,0.23,0.61,0.542,,1.15
column_1_0,1.0287,0.21,4.99,0.0,***,2.8
column_2_0,-1.4298,0.21,-6.71,0.0,***,0.24
column_3_0,1.8226,0.22,8.28,0.0,***,6.19
column_4_0,-0.8295,0.19,-4.31,0.0,***,0.44
column_5_0,0.9584,0.19,5.16,0.0,***,2.61
column_6_0,0.0122,0.16,0.07,0.941,,1.01
column_7_0,0.1329,0.17,0.77,0.442,,1.14
column_8_0,-0.1344,0.17,-0.79,0.43,,0.87
column_9_0,0.0609,0.17,0.36,0.72,,1.06


In [45]:
model3.get_inference(only_IC=True)

{'LL': -371.0019971973777,
 'AIC_ll': 844.0039943947554,
 'BIC_ll': 1068.247406810779}

As you see the reference class is 3 , interpretation stays the same but always by referring to class 3

We can also compare these results with official implementation of `statmodels`

In [43]:
import statsmodels.api as sm
classes=sorted(set(y))
X=sm.add_constant(pd.DataFrame(x))
new_y = np.where(y == classes[len(classes)-1], -1, y)

mlogit_model = sm.MNLogit(new_y, X)
result = mlogit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.818614
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                  600
Model:                        MNLogit   Df Residuals:                      447
Method:                           MLE   Df Model:                          150
Date:                Sat, 06 Apr 2024   Pseudo R-squ.:                  0.4079
Time:                        20:17:59   Log-Likelihood:                -491.17
converged:                       True   LL-Null:                       -829.60
Covariance Type:            nonrobust   LLR p-value:                 6.156e-68
       y=0       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1378      0.226      0.611      0.541      -0.305       0.580
0              1.0287      0.

## **Model Selection**

In [44]:
index_cols=model3.autoselection("backward","BIC_ll",print_message=False)
index_cols

array([ 0,  1,  2,  3,  4, 13, 18, 22, 26, 43])

## **OVR Algorithm**



Logistic OVR is a multiclass classification strategy where you train multiple binary classifiers, each distinguishing between one class and the rest.

**Classifier Training**:
   - For each class:
     - Transform data into binary classification (class vs. rest).
     - Train binary classifier (e.g., logistic regression).

**Classification**:
   - Predict class by selecting the classifier with highest probability.


In [10]:
modelOVR=LogisticRegression(solver="nr",multiclass="ovr")
modelOVR.fit(x,y)[0:5]

algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)


array([[-1.75252879, -1.52746982, -1.82625903, -1.99017291],
       [-0.12607159,  0.58610452,  0.99065173, -1.30534852],
       [-0.78623603, -0.92842607,  1.01896941,  0.94986865],
       [ 1.48922124, -0.98541303,  0.34939119, -0.66088104],
       [-0.75319425,  1.18077461, -0.56564383,  0.10412552]])

As you see we have K columns for B estimator 

In [11]:
modelOVR.predict(x)[0:5]

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]])

In [17]:
modelOVR.proba[0:5]


array([[5.56356354e-02, 5.42091031e-01, 3.85392441e-01, 1.68808924e-02],
       [5.56460760e-02, 2.50315712e-02, 6.88537280e-02, 8.50468625e-01],
       [3.80882686e-02, 2.12180125e-03, 8.28453620e-01, 1.31336310e-01],
       [6.95336242e-02, 2.92077653e-01, 4.38635151e-01, 1.99753572e-01],
       [7.95592741e-01, 4.80155378e-04, 6.11773762e-02, 1.42749727e-01]])

In [19]:
modelOVR.y

array([[1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       ...,
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]])

In [20]:
#nb of folds is 4 so we will run 4 models 
CV_scores=CrossValidation(Class_algorithm=modelOVR,x=x,y=modelOVR.y,metrics_function=accuracy_score,nb_k_fold=4)
print(CV_scores)
np.mean(CV_scores)

algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 6 iterations)
algorithm did  converge under 100 iterations (at 7 iter

0.525

Performance decreased so we will privilege ordinary logistic regression in this case

In [21]:
index_cols=modelOVR.autoselection("forward","BIC_ll",print_message=False)
index_cols

array([ 2,  0,  1,  3,  4, 13, 22, 43, 18, 23])